In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ---------------------------
# 1. Load Data
# ---------------------------

orders = pd.read_csv("orders_data.csv")

# ---------------------------
# 2. Convert Date Columns
# ---------------------------

orders["created_at"] = pd.to_datetime(orders["created_at"].str.replace(" UTC",""))
orders["first_date_order"] = pd.to_datetime(orders["first_date_order"].str.replace(" UTC",""))

# ---------------------------
# 3. Create Cohort & Order Month
# ---------------------------

orders["order_month"] = orders["created_at"].dt.to_period("M")
orders["cohort_month"] = orders["first_date_order"].dt.to_period("M")

# ---------------------------
# 4. Compute Cohort Index (Months Since First Order)
# ---------------------------

orders["order_month_ts"] = orders["order_month"].dt.to_timestamp()
orders["cohort_month_ts"] = orders["cohort_month"].dt.to_timestamp()

orders["cohort_index"] = (
    (orders["order_month_ts"].dt.year - orders["cohort_month_ts"].dt.year) * 12 +
    (orders["order_month_ts"].dt.month - orders["cohort_month_ts"].dt.month)
)

# ---------------------------
# 5. Build Revenue Cohort Table
# ---------------------------

# 🔴 Replace REVENUE_COLUMN with your real revenue column name
revenue_cohort = orders.groupby(
    ["cohort_month", "cohort_index"]
)["REVENUE_COLUMN"].sum().reset_index()

# ---------------------------
# 6. Pivot into Revenue Matrix
# ---------------------------

revenue_matrix = revenue_cohort.pivot_table(
    index="cohort_month",
    columns="cohort_index",
    values="REVENUE_COLUMN"
)

# ---------------------------
# 7. Create Revenue Retention Matrix (% of Month 0)
# ---------------------------

revenue_retention = revenue_matrix.divide(revenue_matrix[0], axis=0)

# ---------------------------
# 8. Plot Revenue Retention Heatmap
# ---------------------------

plt.figure(figsize=(14, 9))
sns.heatmap(
    revenue_retention,
    annot=True,
    fmt=".0%",
    cmap="YlGnBu",
    linewidths=.5,
    linecolor="gray",
    cbar_kws={'label': 'Revenue Retention Rate'}
)
plt.title("Revenue Cohort Analysis", fontsize=16, fontweight='bold')
plt.xlabel("Months Since First Purchase")
plt.ylabel("Cohort Month")
plt.tight_layout()
plt.show()


KeyError: 'Column not found: REVENUE_COLUMN'